# 葫蘆問


- http://koaning.io/monopoly-simulations.html
- https://devpost.com/software/monopoly-simulation
- https://github.com/williamhbell/MonopolySimulation/blob/master/doc/MonopolyWorksheet.pdf
- https://github.com/jm-contreras/monopoly
- [Monopoly as a Markov process](https://faculty.math.illinois.edu/~bishop/monopoly.pdf)

## 規則

http://www.jen-pin.com.tw/doc/2013110810042950

**規則一「先走後跳」**

擲點數後，先前進到所擲的位置，再跳到所擲位置的對應圖案。這是什麼意思？以下以圖形來說明。

![規則1](http://www.jen-pin.com.tw/data/2013110810042950/attachments/image010.gif)
丟出2點，先走到**2驢**的地方>>再跳到**同樣是驢的14**位置。

**規則二「頂位」**

若四個人玩，乙丟3點，先走到3葫蘆，再跳到12葫蘆。可是這時候，丙也丟出3點，則丙會把乙從12葫蘆，**擠回**對應的3葫蘆，稱為「**頂位**」。(要注意，「頂位」發生在跳躍的位置，不是擲出點數的位置哦！)

![規則2](http://www.jen-pin.com.tw/data/2013110810042950/attachments/image012.gif)
丟出3點，先走到**3葫蘆**的地方>>再跳到對應的**12葫蘆**的位置。

**規則三「三打不成」**

如果丁也丟出3點，三個人踩在同一位置上，就稱為「**三打不成**」，只能「**和局**」重新開始

![規則3](http://www.jen-pin.com.tw/data/2013110810042950/attachments/image014.gif)
擲出12點，先走到**12葫蘆**的地方，再跳到相對應的**3葫蘆**的位置，也就是只前進三格@@

---

In [284]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import json

import config

In [4]:
# config
print(config.board_filename)
print(config.n_players)
print(config.verbose)

board47.xlsx
2
{'round': True}


In [174]:
# Logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


### Dice

In [6]:
class Dice(object):
    def __init__(self):

        self.roll_sum = None

    def roll(self):
        '''Roll two fair six-sided die and store the sum of the roll'''

        roll = np.random.choice(np.arange(1, 7), 2)
        self.roll_sum = roll.sum()

### Board

In [7]:
board = pd.read_excel(config.board_filename)
board

,figure,position,dual_position,koujue
0,月兔,1,10.0,浸八成肥、浸九無走
1,驢,2,14.0,驢二、驢七驢咬筆
2,葫蘆,3,12.0,葫蘆六就雙雙層、葫蘆九無走、葫蘆四扒鬚
3,鐵枴李,4,15.0,乞食五入大箍、乞食五雞母肚、乞食七偷溜鯽、乞食十壹算粿
4,雞,5,20.0,雞四肥臀臀、雞十一啄龍鐘、雞五豆仔紅、雞六著雞坐、雞七啄(青)菜、雞八烏先生、雞九天公生、雞...
5,虎,6,18.0,虎七一支鬚、虎八顛倒縮、虎九啃虎草
6,呂洞賓,7,23.0,賓九一杯茶、賓仔雙雙去、賓麼麼仙人化、賓二大腳倉、賓五退黑肚、賓六騎鹿、賓七漏(落)姑屐、賓...
7,魚,8,22.0,鯉半斤入銅貢、鯉崎崎做阿爹、鯉三娘推磨、鯉三娘弄關刀、鯉五鹽酸菜、鯉六曹黑墨、鯉七哥射著鹿、...
8,漢鍾離,9,37.0,肥七一槓風、肥八叫司公、鐘九子、肥三落鹿擔、肥四落龜致、肥五捧鍋仔、肥六趖一目、肥七進二匹、...
9,張果老,11,34.0,掃五一尾呆、掃八一個月、掃九無走、掃三五佮七


In [8]:
board.dtypes

figure            object
position           int64
dual_position    float64
koujue            object
dtype: object

In [9]:
goal = board.loc[board['position'].idxmax(), 'figure']
goal

'南極仙翁'

In [10]:
board_list = []
for index, row in board.iterrows():
    if row['figure'] != goal:
        board_list.append({
            'figure': row['figure'],
            'position': int(row['position']), 
            'dual_position': int(row['dual_position'])
        })

board_list

[{'dual_position': 10, 'figure': '月兔', 'position': 1},
 {'dual_position': 14, 'figure': '驢', 'position': 2},
 {'dual_position': 12, 'figure': '葫蘆', 'position': 3},
 {'dual_position': 15, 'figure': '鐵枴李', 'position': 4},
 {'dual_position': 20, 'figure': '雞', 'position': 5},
 {'dual_position': 18, 'figure': '虎', 'position': 6},
 {'dual_position': 23, 'figure': '呂洞賓', 'position': 7},
 {'dual_position': 22, 'figure': '魚', 'position': 8},
 {'dual_position': 37, 'figure': '漢鍾離', 'position': 9},
 {'dual_position': 34, 'figure': '張果老', 'position': 11},
 {'dual_position': 28, 'figure': '曹國舅', 'position': 13},
 {'dual_position': 42, 'figure': '鐘', 'position': 16},
 {'dual_position': 26, 'figure': '韓湘子', 'position': 17},
 {'dual_position': 30, 'figure': '何仙姑', 'position': 19},
 {'dual_position': 32, 'figure': '劉海', 'position': 21},
 {'dual_position': 35, 'figure': '鶴', 'position': 24},
 {'dual_position': 46, 'figure': '銅錢', 'position': 25},
 {'dual_position': 36, 'figure': '葉子', 'position': 27},


### Space

In [11]:
class Space:
    """Generic space object that initializes the two attributes shared by all spaces: Name and position on the board."""

    def __init__(self, attrib):

        self.name = attrib['name']            # Property name
        self.position = attrib['position']    # Board position

### Game

In [449]:
class Huluwen(object):
    def __init__(self):
        self._history = []
    
    def run_game(self, n_players, board_filename, max_round=np.Inf):
        # Initialize game
        g = Game(n_players, board_filename)
        
        while g.status == 'on':
            
            # Update game round
            g.update_round()
            
            # Define player of turn
            for turn_player in g.players:
                
                # Continue until turn ends
                while True:
                    
                    g.update_index()
                    
                    # Roll the dice
                    g.dice.roll()
                    
                    # Move player
                    turn_player.move(g.dice.roll_sum, g.board)
                    
                    # Check game status
                    if turn_player.position == g._board_max:
                        g.status = 'set'
                        self._history.append(g.log[0])
                        break
                    
                    if turn_player.position == 0:
                        g.status = 'tie'
                        self._history.append(g.log[0])
                        break
                    
                    # Update history
                    self._history.append(g.log[0])
                    
                    logger.debug(g.players_info)
                    
                    # End
                    break
                    
            # Stop if game set
            if g.status in ('set', 'tie'):
                break

            # Stop if exceed max_round
            if g.round > max_round:
                break
            
            
    @property
    def history(self):
        return self._history

In [444]:
class Game(object):
    '''Keeps track of all game pieces.'''
    
    def __init__(self, n_players: int, board_filename: str):
        self.players = []
        self.index = 0
        self.round = 0
        self.board = {}
        self.dice = Dice()
        self.status = 'on'

        self._get_board(board_filename)
        self._get_players(n_players)
        
    def _get_board(self, board_file):
        '''
        Create board game with properties from CSV file in board_file.
        :param str board_file: Filename of CSV with board parameters
        '''

        board_df = pd.read_excel(board_file)
        self.goal_fig = board_df.loc[board_df['position'].idxmax(), 'figure']
        self._board_max = board_df['position'].max()
        
        for index, row in board_df.iterrows():
            if row['figure'] != self.goal_fig:
                self.board[int(row['position'])] = {
                    'figure': row['figure'], 
                    'dual_position': int(row['dual_position']),
                    'occupant': []
                }
                self.board[int(row['dual_position'])] = {
                    'figure': row['figure'], 
                    'dual_position': int(row['position']),
                    'occupant': []
                }
            else:
                self.board[int(row['position'])] = {
                    'figure': row['figure'], 
                    'occupant': []
                }
    
    def _get_players(self, n_players):
        '''
        Create list of 2 to 8 game players.
        :param int n_players: Number of players in game
        '''

        # Ensure number of players requested is legal
        if (n_players < 2) or (6 < n_players):
            raise ValueError('A game must have between 2 to 6 players. You input {} players.'.format(n_players))

        # Create list of players and set number of players remaining
        self.players = [Player(string.ascii_uppercase[p], self._board_max) for p in range(n_players)]
    
    
    def update_index(self):

        self.index += 1
        logger.debug(f'Index: {self.index}')
        
    
    def update_round(self):

        self.round += 1
        
        if config.verbose['round']:
            logger.info('Starting round {round}...'.format(round=self.round))
    
    @property
    def log(self):
        turn_log = {
            'index': self.index,
            'round': self.round,
            'dice_roll': self.dice.roll_sum,
            'status': self.status,
        }
        board_log = [{'position': k,
                      'figure': v['figure'],
                      'occupant': v['occupant'][0].id if v['occupant'] else None }
                     for k, v in g.board.items()]
        return turn_log, board_log

    @property
    def players_info(self):
        return pd.DataFrame([p.info for p in self.players])

In [343]:
class Player(object):
    def __init__(self, player_id, board_max, start_pos=0):
        '''Return a Player object.'''
        self.id = str(player_id)
        self.position = start_pos
        self._board_max = board_max

    def move(self, roll, board):
        '''
        Move player on the board. Update player's position and collect $200 if player passed Go.
        :param int roll: Number of board positions to move
        '''
        
        # 先走後跳
        new_position = self.position + roll
        
        if new_position == self._board_max:
            self.position = new_position
            logger.debug(f'{self.id} => {roll} pos: {new_position} ({board[self.position]["figure"]})')
            return     
        
        # 超過終點要倒退
        if new_position > self._board_max:
            new_position -= (new_position - self._board_max) * 2
        
        old_position = self.position
        logger.debug(old_position)
        new_position = board[new_position].get('dual_position')
        self.position = new_position
        
        # Define current board space
        space = board[self.position]
        logger.debug(f'{self.id} | roll: {roll} | pos: {old_position} => {new_position} | ({space["figure"]})')
        
        current_occupant = space['occupant']
        dual_now_occupant = board[space['dual_position']]['occupant']
        
        if current_occupant:
            # 三打不成
            if dual_now_occupant:
                if old_position != 0:
                    board[old_position]['occupant'] = []
                self.position = 0
            # 頂位
            else:
                dual_position = space['dual_position']
                board[dual_position]['occupant'] = current_occupant
                current_occupant[0].position = dual_position
                space['occupant'].append(self)
        else:
            space['occupant'] = [self]

    @property
    def info(self):
        return {'id': self.id, 'position': self.position}

---

## Run

In [450]:
# logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)

h = Huluwen()
h.run_game(n_players=2, board_filename=config.board_filename)

INFO:root:Starting round 1...
DEBUG:root:Index: 1
DEBUG:root:0
DEBUG:root:A | roll: 9 | pos: 0 => 37 | (漢鍾離)
DEBUG:root:  id  position
0  A        37
1  B         0
DEBUG:root:Index: 2
DEBUG:root:0
DEBUG:root:B | roll: 11 | pos: 0 => 34 | (張果老)
DEBUG:root:  id  position
0  A        37
1  B        34
INFO:root:Starting round 2...
DEBUG:root:Index: 3
DEBUG:root:37
DEBUG:root:A | roll: 9 | pos: 37 => 25 | (銅錢)
DEBUG:root:  id  position
0  A        25
1  B        34
DEBUG:root:Index: 4
DEBUG:root:34
DEBUG:root:B | roll: 11 | pos: 34 => 31 | (龍)
DEBUG:root:  id  position
0  A        25
1  B        31
INFO:root:Starting round 3...
DEBUG:root:Index: 5
DEBUG:root:25
DEBUG:root:A | roll: 7 | pos: 25 => 21 | (劉海)
DEBUG:root:  id  position
0  A        21
1  B        31
DEBUG:root:Index: 6
DEBUG:root:31
DEBUG:root:B | roll: 10 | pos: 31 => 33 | (龜)
DEBUG:root:  id  position
0  A        21
1  B        33
INFO:root:Starting round 4...
DEBUG:root:Index: 7
DEBUG:root:21
DEBUG:root:A | roll: 7 | pos: 2

In [446]:
h.history

[{'dice_roll': 9, 'index': 1, 'round': 1, 'status': 'on'},
 {'dice_roll': 6, 'index': 2, 'round': 1, 'status': 'on'},
 {'dice_roll': 8, 'index': 3, 'round': 2, 'status': 'on'},
 {'dice_roll': 6, 'index': 4, 'round': 2, 'status': 'on'},
 {'dice_roll': 6, 'index': 5, 'round': 3, 'status': 'on'},
 {'dice_roll': 10, 'index': 6, 'round': 3, 'status': 'on'},
 {'dice_roll': 4, 'index': 7, 'round': 4, 'status': 'tie'},
 {'dice_roll': 8, 'index': 8, 'round': 4, 'status': 'tie'}]

In [78]:
45 % 20

5